In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as models
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split

In [34]:
#    transforms.Resize(256),
#    transforms.CenterCrop(224),
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
data = torchvision.datasets.CIFAR10("./", train=True, download=True, transform=preprocess)
test_data = torchvision.datasets.CIFAR10("./", train=False, transform=preprocess)

Files already downloaded and verified


In [35]:
train_size = int(0.9*len(data))
test_size = len(data) - train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size,test_size])
trainloader = torch.utils.data.DataLoader(train_set, batch_size=100,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=100,
                                          shuffle=False)
valloader = torch.utils.data.DataLoader(val_set, batch_size=100,
                                          shuffle=True)

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        return x
    
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs, msg):
    best_accuracy = 0

    for epochs in range(num_epochs):
        for batch_nr, (data, labels) in enumerate(train_loader):
         
            prediction = model.forward(data)

            loss = criterion(prediction, labels)

            loss.backward()

            optimizer.step()

            optimizer.zero_grad()
            
            print(
            f'\rEpoch {epochs+1} [{batch_nr+1}/{len(train_loader)}] - Loss: {loss}',
            end=''
        )
        val_accuracy = 0
        total = 0
        with torch.no_grad():
            for batch_nr, (data, labels) in enumerate(val_loader):
                prediction = model.forward(data)
                _, predicted = torch.max(prediction, 1)
                val_accuracy += (predicted == labels).sum().item()
                total += labels.size(0)
            print(" ",val_accuracy/total, "Val Acc")
            val_accuracy = val_accuracy/total
            if(best_accuracy < val_accuracy):
                best_accuracy = val_accuracy
                print("best model here")
                torch.save(model.state_dict(), "./E1")
def test_model(model, test_loader):
    test_accuracy = 0
    total = 0
    predictList = []
    testList = []
    
    for batch_nr, (data, labels) in enumerate(test_loader):
        prediction = model.forward(data)
        _, predicted = torch.max(prediction, 1)
        for i in range(len(prediction)):
            predictList.append(predicted[i].item())
            testList.append(labels[i].item())
        test_accuracy += (predicted == labels).sum().item()
        total += labels.size(0)
    print(test_accuracy/total, "Test Accuracy")

In [37]:
model = Net()
LEARNING_RATE = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), LEARNING_RATE)
msg = "trainloss leaky and SGD"
trained_model = train_model(model, criterion, optimizer, trainloader, valloader, 10, msg)
model.load_state_dict(torch.load("./E1"))
tested_model = test_model(model, testloader)

Epoch 1 [450/450] - Loss: 2.2975387573242188  0.1034 Val Acc
best model here
Epoch 2 [450/450] - Loss: 2.2819349765777593  0.118 Val Acc
best model here
Epoch 3 [450/450] - Loss: 2.2325534820556648  0.1422 Val Acc
best model here
Epoch 4 [450/450] - Loss: 2.2047097682952882  0.1908 Val Acc
best model here
Epoch 5 [450/450] - Loss: 2.2488265037536623  0.2298 Val Acc
best model here
Epoch 6 [450/450] - Loss: 2.1124379634857178  0.247 Val Acc
best model here
Epoch 7 [450/450] - Loss: 2.0964837074279785  0.2514 Val Acc
best model here
Epoch 8 [450/450] - Loss: 1.9765672683715825  0.2554 Val Acc
best model here
Epoch 9 [450/450] - Loss: 2.0073533058166504  0.2648 Val Acc
best model here
Epoch 10 [450/450] - Loss: 1.9896255731582642  0.2698 Val Acc
best model here
0.2913 Test Accuracy


In [18]:
LEARNING_RATE = 0.001
model_ft = models.alexnet(pretrained=True)

# Do the things required for fine-tuning before training the model
model_ft.last_linear = nn.Sequential(
    nn.Linear(in_features=100, out_features=10),
)

criterion_ft = nn.CrossEntropyLoss()

# Define our optimizer

optimizer_ft = optim.Adam(model_ft.parameters(), LEARNING_RATE, weight_decay=1e-2)

# Train the model
trained_model_ft = train_model(model_ft, criterion_ft, optimizer_ft, trainloader, valloader, 2, "1")

# Test the model



C:\Users\odahl\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\odahl\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\odahl/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Epoch 1 [450/450] - Loss: 1.2906702756881714  0.5384 Val Acc
best model here
Epoch 2 [450/450] - Loss: 1.2389678955078125  0.5678 Val Acc
best model here


In [19]:
model_ft.load_state_dict(torch.load("./E1"))
tested_model = test_model(model_ft, testloader)

0.5724 Test Accuracy


# Accuracy for first task, 0.5801 Test Accuracy

In [23]:
for params in model_ft.parameters():
    params.requires_grad_ = False
    
criterion_fe = nn.CrossEntropyLoss()

# Define our optimizer

optimizer_fe = optim.Adam(model_ft.parameters(), LEARNING_RATE, weight_decay=1e-2)

# Train the model

# Train the model
trained_model_fe = train_model(model_ft, criterion_fe, optimizer_fe, trainloader, valloader, 2, "")

# Test the model
model_ft.load_state_dict(torch.load("./E1"))
tested_model = test_model(model_ft, testloader)

Epoch 1 [450/450] - Loss: 0.9347338676452637  0.6324 Val Acc
best model here
Epoch 2 [450/450] - Loss: 1.0582861900329597  0.64 Val Acc
best model here


NameError: name 'test_loader' is not defined

In [24]:
tested_model = test_model(model_ft, testloader)

0.6392 Test Accuracy
